In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
print(sys.path)
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
import importlib
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from data import HistoricTradeDataHandler
from event import *
from strategy import OptionSignalStrategy
from portfolio import OptionSignalPortfolio
from execution import SimulatedExecutionHandler
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2

['c:\\Users\\Zino\\python-playground\\QuantTools\\EventDriven', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\Zino\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv', '', 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages', 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\Pythonwin']

Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL

In [3]:
os.environ['PROXY_URL']

'http://18.232.166.224:5500/thetadata'

In [2]:

def create_datasate(stocks: list, start: str,interval: str, engine: str = 'yf', timewidth = None, timeframe = None, end: str = datetime.today(), return_object = False ):
    dataset = []
    if engine.lower() == 'yf':
        for stock in stocks:
            start = prev_monday(start)
            data2 = yf.download(stock, start = start, end = end, interval=interval, progress = False)

            dataset.append(PTDataset(stock, data2))
    else:
        for stk in stocks:
            stock = Stock(stk)
            data = stock.spot(ts = True, ts_start = '2018-01-01')
            data.rename(columns = {x:x.capitalize() for x in data.columns}, inplace= True)
            data['Timestamp'] = pd.to_datetime(data['Timestamp'], format = '%Y-%m-%d')
            data2 = data.set_index('Timestamp')
            data2 = data2.asfreq('W', method = 'ffill')
            data2 = data2.fillna(0)
            data2['Next_Day_Open'] = data2.Open.shift(-1)
            data2['EMA'] = ta.ma('ema', data2.Close, length = 21).fillna(0)
            dataset.append(PTDataset(stk, data2))
    return dataset if return_object else data2


In [12]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
BBandsTrend2.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, BBandsTrend2, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.trades()
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,4,191,202,205.878064,182.375999,-94.008257,-0.114155,2024-03-04,2024-03-19,None,15 days,AMD
0,2,191,232,414.887042,392.609985,-44.554114,-0.053694,2024-03-04,2024-05-01,None,58 days,MSFT
0,5,191,249,178.151354,181.649994,17.493201,0.019639,2024-03-04,2024-05-24,None,81 days,AMZN
0,7,197,249,137.509604,174.979996,262.292744,0.272493,2024-03-12,2024-05-24,None,73 days,GOOGL
1,4,203,219,182.667104,163.277998,-77.556422,-0.106144,2024-03-20,2024-04-12,None,23 days,AMD
2,5,220,224,165.005498,146.951999,-90.267494,-0.109411,2024-04-15,2024-04-19,None,4 days,AMD
3,4,230,233,159.636782,145.509995,-56.507149,-0.088493,2024-04-29,2024-05-02,None,3 days,AMD
1,2,233,249,399.051814,427.190002,56.276378,0.070513,2024-05-02,2024-05-24,None,22 days,MSFT
4,4,236,237,156.867127,153.399994,-13.868534,-0.022102,2024-05-07,2024-05-08,None,1 days,AMD
0,5,241,249,188.166279,188.820007,3.268639,0.003474,2024-05-14,2024-05-24,None,10 days,AAPL


In [12]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
MAStrat.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, MAStrat, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.trades()
shorts = tt.trades()[tt.trades()['Size'] < 0]
shorts

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,121.234194,121.360001,-1.006456,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,339.288331,333.670013,11.236635,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...
55,-4,238,239,152.594050,154.289993,-6.783774,-0.011114,2024-05-09,2024-05-10,None,1 days,AMD
56,-4,240,241,150.750519,150.429993,1.282104,0.002126,2024-05-13,2024-05-14,None,1 days,AMD
57,-4,242,243,155.045431,160.919998,-23.498267,-0.037889,2024-05-15,2024-05-16,None,1 days,AMD
59,-4,249,249,160.845069,161.410004,-2.259740,-0.003512,2024-05-24,2024-05-24,None,0 days,AMD


In [17]:
trades = trades[:10]

In [10]:
equity = tt.pf_value_ts()
equity

,AAPL,MSFT,GOOGL,AMD,AMZN,Total
2023-05-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000
2023-05-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000
2023-06-01,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000
2023-06-02,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000
2023-06-05,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000
...,...,...,...,...,...,...
2024-05-20,692.291548,900.277942,815.820238,708.785599,757.180265,3874.355592
2024-05-21,692.291548,900.277942,815.820238,708.785599,752.220304,3869.395631
2024-05-22,692.291548,900.277942,815.820238,708.785599,752.220304,3869.395631
2024-05-23,692.291548,900.277942,815.820238,708.785599,752.220304,3869.395631


In [21]:
#Backtest class
evb_backtest = OptionSignalBacktest(trades) 

In [22]:
signals = evb_backtest.bars.signal_df
signals

,Date,GOOGL,AMD,MSFT,AMZN
0,2023-07-24,2,2,0,0
1,2023-07-25,-1,-1,0,0
2,2023-07-26,1,2,0,0
3,2023-07-27,-1,-1,2,0
4,2023-07-28,0,2,-1,0
5,2023-07-29,0,0,0,0
6,2023-07-30,0,0,0,0
7,2023-07-31,0,-1,0,1
8,2023-08-01,0,2,2,-1
9,2023-08-02,0,-1,-1,0


In [23]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest
evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  GOOGL  AMD  MSFT  AMZN
0 2023-07-24      2    2     0     0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
        Date  GOOGL  AMD  MSFT  AMZN
1 2023-07-25     -1   -1     0     0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
        Date  GOOGL  AMD  MSFT  AMZN
2 2023-07-26      1    2     0     0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Failed to fetch contract for {underlier.ticker} at {time_str}
Processing event: ORDER
Processing event: FILL
        Date  GOOGL  AMD  MSFT  AMZN
3 2023-07-27     -1   -1     2     0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing

In [ ]:
stats.print_stats(20)  # Show the top 20 functions by cumulative time
print(stream.getvalue())

In [17]:
trades = evb_backtest.portfolio.get_trades()
print(trades.to_string())

    EntryPrice  ExitPrice     PnL   ReturnPct  EntryTime   ExitTime  Duration Ticker
0          0.0     1550.0  1550.0         inf 2024-03-04 2024-03-19        15    AMD
1       1950.0     1350.0  -600.0  -30.769231 2024-03-20 2024-04-12        23    AMD
2       1702.0        0.0 -1702.0 -100.000000 2024-04-15 2024-04-19         4    AMD
3       1315.0      338.0  -977.0  -74.296578 2024-03-04 2024-05-01        58   MSFT
4       1765.0     1102.0  -663.0  -37.563739 2024-04-29 2024-05-02         3    AMD
5        650.0      595.0   -55.0   -8.461538 2024-05-07 2024-05-08         1    AMD
6       1188.0     1300.0   112.0    9.427609 2024-05-16 2024-05-24         8    AMD
7        652.0     1040.0   388.0   59.509202 2024-05-02 2024-05-24        22   MSFT
8       1405.0     1209.0  -196.0  -13.950178 2024-03-04 2024-05-24        81   AMZN
9        520.0     2138.0  1618.0  311.153846 2024-03-12 2024-05-24        73  GOOGL
10       100.0        0.0  -100.0 -100.000000 2024-05-14 2024-05-

In [80]:
#Get all holdings
holdings = evb_backtest.get_all_holdings()
print(holdings.to_string())

               AMD    MSFT   AMZN   GOOGL   AAPL      cash  commission     total
datetime                                                                        
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05  3680.0     0.0    0.0     0.0    0.0   96320.0         0.0  100000.0
2024-03-05  3680.0  1624.0    0.0     0.0    0.0   94696.0         0.0  100000.0
2024-03-05  3680.0  1624.0  416.0     0.0    0.0   94280.0         0.0  100000.0
2024-03-06  4596.0  1402.0    0.0     0.0    0.0   94280.0         0.0  100278.0
2024-03-07  4804.0  1770.0    0.0     0.0    0.0   94280.0         0.0  100854.0
2024-03-08  4630.0  1880.0    0.0     0.0    0.0   94280.0         0.0  100790.0
2024-03-11  3746.0     0.0  380.0     0.0    0.0   94280.0         0.0   98406.0
2024-03-12  3310.0  1990.0  378.0     0.0    0.0   94280.0         0.0   99958.0
2024-03-13  3178.0  2096.0  

In [36]:
positions = evb_backtest.get_all_positions()
print(positions.to_string())

                             AMD                   AAPL                   MSFT                   GOOGL
datetime                                                                                              
2024-03-04                  None                   None                   None                    None
2024-03-04                  None                   None                   None                    None
2024-03-05  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-06  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-07  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-08  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-11  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-12  AMD-20240816-270.0-C                   None  MSFT-20240816-48

In [64]:
evb_backtest.portfolio.all_positions

[{'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20240920-240.0-C',
   'entry_price': 2185.0},
  'MSFT': {'quantity': 2,
   'option': 'MSFT-20250117-540.0-C',
   'entry_price': 778.0},
  'AMZN': {'quantity': 2,
   'option': 'AMZN-20250321-205.0-C',
   'entry_price': 0.0},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-05 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20

In [7]:
from pprint import pprint

print('All positions:')
pprint(portfolio.all_positions)

print('\nCurrent holdings:')
pprint(portfolio.current_holdings)

print('\nCurrent positions:')
pprint(portfolio.current_positions)

All positions:
[{'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': '20240226'},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')}]

Current holdings:
{'AAPL': 0.0,
 'AMD': 0.0,
 'AMZN': 0.0,
 'GOOGL': 0.0,
 'MSFT': 0.0,
 'cash': 100000,
 'commission': 0.0,
 'total': 100000}

Current positions:
{'AAPL': {'option': N

In [14]:
def generate_option_to_buy(underlier: Stock, contract_time):
        """
        Buy an option based on the underlier.
        """
        time = contract_time
        next_day_time = time + pd.DateOffset(days=1)
        print(time, next_day_time)
        option_spot = underlier.spot(ts=True, ts_start = time, ts_end = next_day_time)
        option_spot = option_spot.iloc[0] 
        stock_price = option_spot['open']#use open price as spot price on the assumption of making trades at start of day
        oom_benchmark = 0.1#10% out of the money 
        expiry_benchmark = time + pd.DateOffset(months=5)
        oom_price = stock_price * (1 + oom_benchmark)
        time_str = time.strftime("%Y%m%d")
        contracts = list_contracts(underlier.ticker, time_str)
        print(contracts)
        contracts = contracts[contracts['right'] == 'C'] 
        
        
        #Filter out contracts that are out of the money
        contracts = contracts[contracts['strike'] >= oom_price]
        
        print('comparing expiry')
        print(type(expiry_benchmark))
        print(type(contracts['expiration']))
        #filter out contracts that are  below the expiry benchmark
        contracts = contracts[pd.to_datetime(contracts['expiration'], format="%Y%m%d") >= expiry_benchmark]
        
        #select a random contract to buy
        contract = contracts.sample(n=1); 
        
        return contract
                

In [8]:
appl = Stock('AAPL')
c_time = pd.to_datetime('2024-05-06', format="%Y-%m-%d")
c_time_next = c_time + pd.DateOffset(days=1)
aapl_spot = appl.spot(ts=True, ts_start = c_time, ts_end = c_time_next)
aapl_spot.iloc[0]['open']


182.35000610351562

In [15]:
option = generate_option_to_buy(appl, c_time)
option

2024-05-06 00:00:00 2024-05-07 00:00:00
      root  expiration  strike right
0     AAPL    20241018   220.0     C
1     AAPL    20250919   215.0     C
2     AAPL    20241115   220.0     C
3     AAPL    20250321   220.0     C
4     AAPL    20250321   220.0     P
...    ...         ...     ...   ...
1060  AAPL    20240621   220.0     C
1061  AAPL    20240621   220.0     P
1062  AAPL    20240719   220.0     C
1063  AAPL    20250620   215.0     C
1064  AAPL    20240816   220.0     C

[1065 rows x 4 columns]
comparing expiry
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.series.Series'>


,root,expiration,strike,right
456,AAPL,20250117,235.0,C


In [9]:
list_contracts('MSFT', '20240304')

,root,expiration,strike,right
0,MSFT,20250620,610.0,C
1,MSFT,20240621,485.0,C
2,MSFT,20240621,355.0,P
3,MSFT,20240621,355.0,C
4,MSFT,20240719,355.0,C
...,...,...,...,...
1307,MSFT,20250117,610.0,C
1308,MSFT,20250117,480.0,C
1309,MSFT,20240315,485.0,C
1310,MSFT,20261218,600.0,C


In [4]:
eod= retrieve_option_ohlc('AMD', exp = '20250117', strike=240.0, right = 'C', start_date = '20240304', end_date = '20250117')
print(eod)

     ms_of_day   open   high    low  close  volume  count       date
5     52200000  30.65  30.65  30.65  30.65       1      1 2024-03-04
11    52200000   0.00   0.00   0.00   0.00       0      0 2024-03-05
17    52200000  31.95  31.95  30.93  30.93       5      5 2024-03-06
23    52200000  33.24  33.24  32.08  32.20       5      4 2024-03-07
29    52200000  31.92  31.92  31.38  31.60       3      3 2024-03-08
..         ...    ...    ...    ...    ...     ...    ...        ...
929   52200000   1.38   1.38   1.38   1.38       1      1 2024-10-14
935   52200000   0.85   0.85   0.85   0.85      10      1 2024-10-15
941   52200000   0.00   0.00   0.00   0.00       0      0 2024-10-16
947   52200000   0.83   0.83   0.83   0.83       5      2 2024-10-17
953   52200000   0.73   0.73   0.73   0.73       1      1 2024-10-18

[159 rows x 8 columns]


In [22]:
import requests
url = "http://127.0.0.1:25510/v2/hist/option/ohlc"
querystring = {"end_date":'20241020', "root": 'AMD', "use_csv": "true", "exp": '20250117', "ivl": 3600000, "right": 'C', "start_date": '20240304', "strike": 240.0 * 1000}
headers = {"Accept": "application/json"}
response = requests.get(url, headers=headers, params=querystring)
print(response.status_code, response.text, )

473 For input string


In [46]:
import pandas as pd
options = [{'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 120},
  'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'},
  'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': '2024-03-05',},
 {'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 140},
  'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'},
  'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': '2024-03-06'},
 {'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C'},
  'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'},
  'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': '2024-03-07'},
 {'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C'},
  'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'},
  'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': '2024-03-08'}]

trade_data = []
option_ids = []
for option in options: 
  for option_data_key, option_data_value in option.items(): 
    if option_data_key != 'datetime':
      if option_data_value['option'] is not None:
        print(option_data_key, option_data_value)
        option_ids.append(option_data_value['option'])
        option_id = option_data_value['option']
        fist_entry_price_occur = next((data for data in options if data[option_data_key]['option'] == option_id and 'entry_price' in data[option_data_key]), None)
        entry_price = fist_entry_price_occur[option_data_key]['entry_price'] if fist_entry_price_occur is not None else None
        first_exit_price_occur = next((data for data in options if data[option_data_key]['option'] == option_id and 'exit_price' in data[option_data_key]), None)
        exit_price = first_exit_price_occur[option_data_key]['exit_price'] if first_exit_price_occur is not None else None
        pnl = exit_price - entry_price if exit_price is not None and entry_price is not None else None
        



AMD {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 120}
{'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 120}, 'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'}, 'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'}, 'GOOGL': {'quantity': 0.0, 'option': None}, 'AAPL': {'quantity': 0.0, 'option': None}, 'datetime': '2024-03-05'}
MSFT {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'}
None
AMZN {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'}
None
AMD {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 140}
{'AMD': {'quantity': 2, 'option': 'AMD-20240816-270.0-C', 'entry_price': 120}, 'MSFT': {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'}, 'AMZN': {'quantity': 2, 'option': 'AMZN-20250321-205.0-C'}, 'GOOGL': {'quantity': 0.0, 'option': None}, 'AAPL': {'quantity': 0.0, 'option': None}, 'datetime': '2024-03-05'}
MSFT {'quantity': 2, 'option': 'MSFT-20240816-480.0-C'}
None
AMZN {'quantity': 2, 'option': 'AMZN-20250321

In [40]:
{}.items() 


SyntaxError: invalid syntax (163794015.py, line 1)